## RNN

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential, Model
import optuna
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lynxx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lynxx\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Preprocessing module

In [2]:
# Define stop_words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [3]:
def preprocess_text(text):
    text = text.lower() # convert text to lowercase
    text = re.sub(r'\b(u\.s\.|us)\b', 'usa', text, flags=re.IGNORECASE)  # replace "U.S." or "US" with "usa"
    text = re.sub(r'<[^>]+>', '', text)  # remove HTML tags
    text = re.sub(r'&\w+;', '', text)  # remove HTML entities
    text = re.sub(r'[^a-z\s]', '', text)  # remove numbers and punctuation (keep only letters and spaces)
    words = text.split()
    words = [word for word in words if word not in stop_words] # remove stop-words, tokenization
    words = [lemmatizer.lemmatize(word) for word in words] # lemmatization
    return ' '.join(words) # combining words into a string

In [4]:
# Loading the data
train_df = pd.read_csv('../data/agn_train.csv')
test_df = pd.read_csv('../data/agn_test.csv')

In [5]:
%%time

# Applying preprocessing to titles and descriptions
train_df['clean_text'] = (train_df['Title'] + ' ' + train_df['Description']).apply(preprocess_text)
test_df['clean_text'] = (test_df['Title'] + ' ' + test_df['Description']).apply(preprocess_text)

# Tokenization
tokenizer = Tokenizer(num_words=5000)  # Limit the number of words
tokenizer.fit_on_texts(train_df['clean_text'])  # Tokenize based on the training data

# Tokenizing titles and descriptions
X_train_title_seq = tokenizer.texts_to_sequences(train_df['Title'])
X_test_title_seq = tokenizer.texts_to_sequences(test_df['Title'])

X_train_description_seq = tokenizer.texts_to_sequences(train_df['Description'])
X_test_description_seq = tokenizer.texts_to_sequences(test_df['Description'])

# Determine the maximum sequence length
max_length_titles = max([len(x) for x in X_train_title_seq])
max_length_descriptions = max([len(x) for x in X_train_description_seq])

# Padding sequences to make them of equal length
X_train_title_pad = pad_sequences(X_train_title_seq, maxlen=max_length_titles)
X_test_title_pad = pad_sequences(X_test_title_seq, maxlen=max_length_titles)

X_train_description_pad = pad_sequences(X_train_description_seq, maxlen=max_length_descriptions)
X_test_description_pad = pad_sequences(X_test_description_seq, maxlen=max_length_descriptions)

# Assign labels for the training data
y_train = train_df['Class Index'].values - 1  # Adjust class indices to [0, 1, 2, 3]

# Assign labels for the test data
y_test = test_df['Class Index'].values - 1  # Adjust class indices to [0, 1, 2, 3]

# Loading pre-trained GloVe embeddings
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Creating an embedding matrix for the words in the tokenizer
embedding_matrix = np.zeros((5000, 100))  # input_dim=5000, output_dim=100 (GloVe vector size)
for word, i in tokenizer.word_index.items():
    if i < 5000:  # Only use the top 5000 words
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Using pre-trained GloVe embeddings
embedding_layer = Embedding(input_dim=5000, 
                            output_dim=100, 
                            weights=[embedding_matrix],  # Passing the pre-trained embedding matrix
                            trainable=False)  # Embeddings are not trainable

CPU times: total: 29.5 s
Wall time: 30.9 s


### Training module

In [8]:
%%time

from tensorflow.keras.layers import LSTM

# Input for titles
title_input = Input(shape=(max_length_titles,), name='title_input')
title_embedding = Embedding(input_dim=5000, output_dim=100, weights=[embedding_matrix], trainable=False)(title_input)
title_lstm = LSTM(128, return_sequences=False)(title_embedding)  # LSTM layer for title

# Input for descriptions
description_input = Input(shape=(max_length_descriptions,), name='description_input')
description_embedding = Embedding(input_dim=5000, output_dim=100, weights=[embedding_matrix], trainable=False)(description_input)
description_lstm = LSTM(128, return_sequences=False)(description_embedding)  # LSTM layer for description

# Merging the title and description representations
merged = Concatenate()([title_lstm, description_lstm])

# Fully connected layers after merging
dense_1 = Dense(128, activation='relu')(merged)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(4, activation='softmax')(dense_2)  # 4 classes

# Create the model
model = Model(inputs=[title_input, description_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the model with early stopping
model.fit([X_train_title_pad, X_train_description_pad], 
          y_train, 
          validation_split=0.1, epochs=10, batch_size=32, verbose=1, 
          callbacks=[early_stopping])

Epoch 1/10
3375/3375 ━━━━━━━━━━━━━━━━━━━━ 105s 30ms/step - accuracy: 0.8680 - loss: 0.3847 - val_accuracy: 0.8948 - val_loss: 0.2781
Epoch 2/10
  86/3375 ━━━━━━━━━━━━━━━━━━━━ 1:31 28ms/step - accuracy: 0.8995 - loss: 0.2753

KeyboardInterrupt: 

### Hyperparameter selection module

In [10]:
%%time

# Defining hyperparameters with Optuna
def model_builder(trial):
    # Hyperparameters to optimize
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    lstm_units = trial.suggest_int('lstm_units', 64, 256)  # Number of LSTM units
    num_units = trial.suggest_int('num_units', 64, 512)  # For dense layers

    # Input for titles
    title_input = Input(shape=(max_length_titles,), name='title_input')
    title_embedding = Embedding(input_dim=5000, output_dim=100, 
                                weights=[embedding_matrix], trainable=False)(title_input)
    title_lstm = LSTM(lstm_units, return_sequences=False)(title_embedding)  # LSTM for title
    
    # Input for descriptions
    description_input = Input(shape=(max_length_descriptions,), name='description_input')
    description_embedding = Embedding(input_dim=5000, output_dim=100, 
                                      weights=[embedding_matrix], trainable=False)(description_input)
    description_lstm = LSTM(lstm_units, return_sequences=False)(description_embedding)  # LSTM for description

    # Merging the title and description representations
    merged = Concatenate()([title_lstm, description_lstm])
    
    # Fully connected layers
    dense_1 = Dense(num_units, activation='relu')(merged)
    dropout_1 = Dropout(dropout_rate)(dense_1)
    output = Dense(4, activation='softmax')(dropout_1)  # 4 classes

    # Create the model
    model = Model(inputs=[title_input, description_input], outputs=output)
    
    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Function for model evaluation
def objective(trial):
    model = model_builder(trial)

    # Train the model with the optimized hyperparameters
    model.fit([X_train_title_pad, X_train_description_pad], y_train, 
              validation_split=0.1, epochs=5, batch_size=32, verbose=0)

    # Evaluate the model's accuracy
    loss, accuracy = model.evaluate([X_test_title_pad, X_test_description_pad], y_test, verbose=0)
    return accuracy  # Optimizing for accuracy

# Running Optuna to search for the best hyperparameters
study = optuna.create_study(direction='maximize')  # Maximize accuracy
study.optimize(objective, n_trials=10)  # Number of optimization trials

# Print the best trial results
print(study.best_trial)

[I 2024-10-21 20:54:46,184] A new study created in memory with name: no-name-95fe40ae-d244-4430-bf63-6e8b1dbbca7b
[W 2024-10-21 21:00:22,608] Trial 0 failed with parameters: {'dropout_rate': 0.38001607424830175, 'learning_rate': 0.0001476066959409187, 'lstm_units': 140, 'num_units': 405} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\lynxx\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<timed exec>", line 43, in objective
  File "C:\Users\lynxx\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lynxx\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\trainer.py", line 320, in fit
    logs = self.train_function(iterator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lynxx\anaconda3\Lib\s

KeyboardInterrupt: 

### Testing module

In [11]:
%%time

# Load the best model using the best hyperparameters found by Optuna
best_trial = study.best_trial
best_params = best_trial.params

# Build the model with the best hyperparameters
model = model_builder(best_trial)

# Train the model on the training data
model.fit([X_train_title_pad, X_train_description_pad], y_train, 
          validation_split=0.1, epochs=5, batch_size=32, verbose=0)

# Make predictions on the test data
y_pred = model.predict([X_test_title_pad, X_test_description_pad])
y_pred_classes = y_pred.argmax(axis=1)  # Convert predicted probabilities to predicted classes

# Calculate accuracy
test_accuracy = (y_pred_classes == y_test).mean()
print(f'Test Accuracy: {test_accuracy:.4f}')

# Generate classification report (Precision, Recall, F1-Score)
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))

# Create a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1, 2, 3], yticklabels=[0, 1, 2, 3])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

ValueError: No trials are completed yet.